In [ ]:
import os
import numpy as np
from PIL import Image, ImageFile
from skimage import io
import matplotlib.pyplot as plt
import psycopg2
import tifffile
from scipy import misc
from tifffile import TiffFile
from tifffile.tifffile import TIFF_DECOMPESSORS
from configparser import ConfigParser
from batch_tiff_to_jpeg import tiff_to_jpg
from psycopg2.extras import DictCursor
from visualization import get_connection, get_gta_image_jpg_dir
import progressbar
import random
from os import path
import math
import humanize
JPEG_DIR = 'D:/GTAV_extraction_output/rgb-jpeg'
JPEG_DIR_2 = 'Y:/GTA-jpg'
JPEG_DIRS = [JPEG_DIR, JPEG_DIR_2]
TIFF_DIR = 'D:/GTAV_extraction_output'

In [ ]:
# threaded connection pooling
from psycopg2.pool import PersistentConnectionPool
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

conn_pool = None

def get_connection_pooled():
    """
    :rtype: connection
    """
    global conn_pool
    if conn_pool is None:
        CONFIG = ConfigParser()
        CONFIG.read("gta-postprocessing.ini")
        minimum = 1
        maximum = 28
        conn_pool = PersistentConnectionPool(minimum, maximum, CONFIG["Postgres"]["db"], cursor_factory=DictCursor)
    conn = conn_pool.getconn()
    conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
    return conn

#### selecting images metadata, grouping by position

In [ ]:
conn = get_connection()
cur = conn.cursor()
lag_interval_min = 30

# for each position, select first time and last time car was in this position
cur.execute("SELECT DISTINCT COUNT(camera_pos) AS count FROM snapshots")

position_count = cur.fetchone()['count']
# cur = conn.cursor('server_cursor_{}'.format(random.randint(0, 1e9)))
cur = conn.cursor()
print(cur.itersize)

cur.execute("""SELECT \
        ARRAY[st_x(camera_pos), st_y(camera_pos), st_z(camera_pos)] as position, \
    camera_pos, min(timestamp) AS minimum, max(timestamp) AS maximum, \
    min(imagepath), min(proj_matrix), \
    count(camera_pos) as count \
    FROM snapshots \
    GROUP BY camera_pos \
    ORDER BY min(timestamp) \
    """)

positions = {}
print('positions_total: {}'.format(position_count))

widgets = [progressbar.Percentage(), ' ', progressbar.Counter(), ' ', progressbar.Bar(), ' ', progressbar.FileTransferSpeed()]
pbar = progressbar.ProgressBar(widgets=widgets, max_value = cur.rowcount).start()

for i, row in enumerate(cur):
    pbar.update(i)
    positions[tuple(row['position'])] = {
        'min': row['minimum'],
        'max': row['maximum'],
        'time': row['maximum'] - row['minimum'],
        'position': row['position'],
        'count': row['count'],
        'index': i,
    }
pbar.finish()

print('total {} positions'.format(len(positions)))
# timespans = {key:pos['max'] - pos['min'] for key, pos in positions.items()}


#### selecting images metadata, grouping by projection matrix

In [ ]:
conn = get_connection()
cur = conn.cursor()
lag_interval_min = 30

# for each position, select first time and last time car was in this position
cur.execute("SELECT DISTINCT COUNT(camera_pos) AS count FROM snapshots")

position_count = cur.fetchone()['count']
# cur = conn.cursor('server_cursor_{}'.format(random.randint(0, 1e9)))
cur = conn.cursor()
print(cur.itersize)

# somehow, grouping by proj matrix shows me same images, but grouping by camera pos does not
cur.execute("""SELECT \
        min(ARRAY[st_x(camera_pos), st_y(camera_pos), st_z(camera_pos)]) as position, \
    min(camera_pos), min(timestamp) AS minimum, max(timestamp) AS maximum, \
    min(imagepath), \
    proj_matrix, \
    count(proj_matrix) as count \
    FROM snapshots \
    GROUP BY proj_matrix \
    ORDER BY min(timestamp) \
    """)

projections = {}
print('positions_total: {}'.format(position_count))

widgets = [progressbar.Percentage(), ' ', progressbar.Counter(), ' ', progressbar.Bar(), ' ', progressbar.FileTransferSpeed()]
pbar = progressbar.ProgressBar(widgets=widgets, max_value = cur.rowcount).start()

for i, row in enumerate(cur):
    pbar.update(i)
#    print(tuple(tuple(x) for x in row['proj_matrix']))
    projections[tuple(tuple(x) for x in row['proj_matrix'])] = {
        'min': row['minimum'],
        'max': row['maximum'],
        'time': row['maximum'] - row['minimum'],
        'position': row['position'],
        'proj_matrix': row['proj_matrix'],
        'count': row['count'],
        'index': i,
    }
pbar.finish()

print('total {} projections'.format(len(projections)))
# timespans = {key:pos['max'] - pos['min'] for key, pos in positions.items()}


#### display images per position

In [ ]:
offset = 38100
stride = 1
for position, data in sorted(positions.items(), key=lambda x:x[1]['index'])[offset::stride]:
    i = data['index']
    cur = conn.cursor()
    print(position)
# fucking python has only 1e-12 float position, which is not enough for postgres
    cur.execute("""SELECT \
        imagepath, \
        ARRAY[st_x(camera_pos), st_y(camera_pos)::double precision, st_z(camera_pos)::double precision] as position, \
        camera_pos \
    FROM snapshots \
      WHERE st_distance(camera_pos, ST_GeomFromText('POINT({} {} {})')) < 1e-9 \
    """.format(position[0], position[1], position[2]))

    print("i: {}, time: {}, same position images: {}".format(i, data['min'], cur.rowcount))
    for row in cur:
        img_path = path.join(JPEG_DIR, 'info-' + row['imagepath'] + '-0.jpg')
        if not path.exists(img_path):
            print('image not found')
            continue
        plt.figure(figsize=(15,15))
        img = Image.open(img_path)
        plt.imshow(img)
        plt.show()


#### display images per projection matrix

In [ ]:
offset = 39700
stride = 1
for projection, data in sorted(projections.items(), key=lambda x:x[1]['index'])[offset::stride]:
    i = data['index']
    cur = conn.cursor()
    print(projection)
# fucking python has only 1e-12 float position, which is not enough for postgres
    cur.execute("""SELECT \
        imagepath, \
        ARRAY[st_x(camera_pos), st_y(camera_pos)::double precision, st_z(camera_pos)::double precision] as position, \
        camera_pos \
    FROM snapshots \
      WHERE proj_matrix::NUMERIC[] = ARRAY{} \
    """.format(data['proj_matrix']))

    print("i: {}, time: {}, same proj_matrix images: {}".format(i, data['min'], cur.rowcount))
    for row in cur:
        img_path = path.join(JPEG_DIR, 'info-' + row['imagepath'] + '-0.jpg')
        if not path.exists(img_path):
            print('image not found')
            continue
        plt.figure(figsize=(15,15))
        img = Image.open(img_path)
        plt.imshow(img)
        plt.show()


In [ ]:
from decimal import *
print(float("431.63467407226560000"))
print(Decimal('431.63467407226560000'))
print(float("-55309.14581298830000000"))
              55309.1458129883
# float('431.63467')

In [ ]:
sorted(positions.items(), key=lambda x:x[1]['min'])

#### showing proj_image histogram

In [ ]:
keys = [x['index'] for key, x in positions.items()]
values = [x['count'] for key, x in positions.items()]
plt.plot(keys, values)
plt.title('count of images per position')
plt.grid(True)
plt.show()

keys = [x['index'] for key, x in projections.items()]
values = [x['count'] for key, x in projections.items()]
plt.plot(keys, values)
plt.title('count of images per projection')
plt.grid(True)
plt.show()

In [ ]:
keys = [x['index'] for key, x in positions.items()]
values = [math.log(x['count']) for key, x in positions.items()]
plt.plot(keys, values)
plt.title('logarithmed count of images per position')
plt.grid(True)
plt.show()

keys = [x['index'] for key, x in projections.items()]
values = [math.log(x['count']) for key, x in projections.items()]
plt.plot(keys, values)
plt.title('logarithmed count of images per projection')
plt.grid(True)
plt.show()

In [ ]:
keys = [x['index'] for key, x in positions.items()]
values = [(x['max'] - x['min']).seconds / 60 for key, x in positions.items()]
plt.plot(keys, values)
plt.title('minutes in same position')
plt.grid(True)
plt.show()

keys = [x['index'] for key, x in projections.items()]
values = [(x['max'] - x['min']).seconds / 60 for key, x in projections.items()]
plt.plot(keys, values)
plt.title('minutes in same projection')
plt.grid(True)
plt.show()


#### showing glitched by positions

In [ ]:
filtered = {key: x for key, x in positions.items() if (((x['max'] - x['min']).seconds / 60) > lag_interval_min)}
keys = [x['index'] for key, x in filtered.items()]
values = [x['count'] for key, x in filtered.items()]
plt.plot(keys, values)
plt.grid(True)
plt.show()

filtered = {key: x for key, x in projections.items() if (((x['max'] - x['min']).seconds / 60) > lag_interval_min)}
keys = [x['index'] for key, x in filtered.items()]
values = [x['count'] for key, x in filtered.items()]
plt.plot(keys, values)
plt.grid(True)
plt.show()


#### showing glitched by projections

In [ ]:
filtered = {key: x for key, x in projections.items() if (((x['max'] - x['min']).seconds / 60) > lag_interval_min)}
total_size = 0

widgets = [progressbar.Percentage(), ' ', progressbar.Counter(), ' ', progressbar.Bar(), ' ', progressbar.FileTransferSpeed()]
pbar = progressbar.ProgressBar(widgets=widgets, max_value = len(filtered)).start()
j = 0
for projection, data in filtered.items():
    j += 1
    pbar.update(j)
    i = data['index']
    cur = conn.cursor()
    cur.execute("""SELECT \
        imagepath, \
        ARRAY[st_x(camera_pos), st_y(camera_pos)::double precision, st_z(camera_pos)::double precision] as position, \
        camera_pos \
    FROM snapshots \
      WHERE proj_matrix::NUMERIC[] = ARRAY{} \
    """.format(data['proj_matrix']))

#    print("i: {}, from: {}, to: {}, same proj_matrix images: {}".format(i, data['min'], data['max'], cur.rowcount))
    pbar1 = progressbar.ProgressBar(widgets=widgets, max_value = cur.rowcount).start()
    k = 0
    for row in cur:
        k += 1
        pbar1.update(k)
        img_path = path.join(JPEG_DIR, 'info-' + row['imagepath'] + '-0.jpg')
        if not path.exists(img_path):
            continue
        total_size += os.path.getsize(img_path)
        img_path = path.join(TIFF_DIR, 'info-' + row['imagepath'] + '.tiff')
        if not path.exists(img_path):
            continue
        total_size += os.path.getsize(img_path)
        img_path = path.join(TIFF_DIR, 'info-' + row['imagepath'] + '-depth.tiff')
        if not path.exists(img_path):
            continue
        total_size += os.path.getsize(img_path)
        img_path = path.join(TIFF_DIR, 'info-' + row['imagepath'] + '-stencil.tiff')
        if not path.exists(img_path):
            continue
        total_size += os.path.getsize(img_path)
    pbar1.finish()
pbar.finish()
        
print('total size is {}'.format(humanize.naturalsize(total_size)))

In [ ]:
total_size

In [ ]:
filtered = {key: x for key, x in projections.items() if (((x['max'] - x['min']).seconds / 60) > lag_interval_min)}
for projection, data in sorted(filtered.items(), key=lambda x:x[1]['index']):
    i = data['index']
    cur = conn.cursor()
    print(projection)
# fucking python has only 1e-12 float position, which is not enough for postgres
    cur.execute("""SELECT \
        imagepath, \
        ARRAY[st_x(camera_pos), st_y(camera_pos)::double precision, st_z(camera_pos)::double precision] as position, \
        camera_pos \
    FROM snapshots \
      WHERE proj_matrix::NUMERIC[] = ARRAY{} \
      LIMIT 10 \
    """.format(data['proj_matrix']))

    print("i: {}, time: {}, same proj_matrix images: {}".format(i, data['min'], cur.rowcount))
    for row in cur:
        img_path = path.join(JPEG_DIR, 'info-' + row['imagepath'] + '-0.jpg')
        if not path.exists(img_path):
            print('image not found')
            continue
        plt.figure(figsize=(15,15))
        img = Image.open(img_path)
        plt.imshow(img)
        plt.show()


## actually deleting things

In [ ]:
from joblib import Parallel, delayed
from psycopg2.pool import PoolError
import time

filtered = {key: x for key, x in projections.items() if (((x['max'] - x['min']).seconds / 60) > lag_interval_min)}
treshold = 3000
jpeg_names = ['info-{}-0.jpg']
tiff_names = ['info-{}.tiff', 'info-{}-depth.tiff', 'info-{}-stencil.tiff']


def delete_image(row):
    global counter
    counter += 1
    pbar.update(counter)
    for DIR in JPEG_DIRS:
        for name in jpeg_names:
            img_path = path.join(DIR, name.format(row['imagepath']))
            if not path.exists(img_path):
                continue
            os.remove(img_path)
    for name in tiff_names:
        img_path = path.join(TIFF_DIR, name.format(row['imagepath']))
        if not path.exists(img_path):
            continue
        os.remove(img_path)
    # n_start = time.time()
    while True:
        try:
            conn2 = get_connection_pooled()
            break
        except PoolError:
            print('waiting for free connection')
            time.sleep(0.5)
            
    cur2 = conn2.cursor()
    cur2.execute('DELETE FROM snapshots WHERE snapshot_id = {}'.format(row['snapshot_id']))
    # n_end = time.time()
    # print('sql time to execute: {}'.format(n_end - n_start))


for projection, data in sorted(filtered.items(), key=lambda x:x[1]['index']):
    i = data['index']
    cur = conn.cursor()
# fucking python has only 1e-12 float position, which is not enough for postgres
    cur.execute("""SELECT \
        snapshot_id, \
        imagepath, \
        ARRAY[st_x(camera_pos), st_y(camera_pos)::double precision, st_z(camera_pos)::double precision] as position, \
        camera_pos \
    FROM snapshots \
      WHERE proj_matrix::NUMERIC[] = ARRAY{} \
    """.format(data['proj_matrix']))
    
    if cur.rowcount < treshold:
        continue
    
    rows = cur.fetchall()
    cur.close()
            
    conn_pool.closeall()
    conn_pool = None

    print("i: {}, time: {}, same proj_matrix images: {}, going to delete them"
          .format(i, data['min'], len(rows)))
    pbar = progressbar.ProgressBar(widgets=widgets, max_value = cur.rowcount).start()
    counter = 0
    workers = 20
    Parallel(n_jobs=workers, backend='threading')(delayed(delete_image)(row) for row in rows)
    # for row in rows:
    #     delete_image(row)
    pbar.finish()


In [ ]:
[r['snapshot_id'] for r in rows]

In [ ]:
row = rows[10]
print(row)
for DIR in JPEG_DIRS:
    for name in jpeg_names:
        img_path = path.join(DIR, name.format(row['imagepath']))
        if not path.exists(img_path):
            print('{} not exists'.format(img_path))
            continue
        os.remove(img_path)
for name in tiff_names:
    img_path = path.join(TIFF_DIR, name.format(row['imagepath']))
    if not path.exists(img_path):
        print('{} not exists'.format(img_path))
        continue
    os.remove(img_path)
